In [142]:
#!/usr/bin/env python3
"""
Module Docstring
"""

__author__ = "Your Name"
__version__ = "0.1.0"
__license__ = "MIT"

import argparse
import os
from logzero import logger
from espn_api.football import League


POSITIONS = ["QB", "WR", "TE", "RB"]

""" Main entry point of the app """
logger.info("hello world from the notebook")
logger.info(args)


# Get environment variables
swid = os.getenv('SWID')
espn_s2 = os.environ.get('ESPN_S2')

if swid == None or espn_s2 == None:
    print("SWID or ESPN_S2 environment variables empty. Exiting...")

league = League(league_id=761056, year=2022, espn_s2=espn_s2,swid=swid)


## Get Box Scores, pass in week 
week = 2
if args.week != None:
    week = args.week


print("Week: ", week)
box_scores = league.box_scores(2)
for matchup in box_scores:
    print(matchup.home_team.team_name )

[I 220924 17:05:44 1843149761:19] hello world from the notebook
[I 220924 17:05:44 1843149761:20] Namespace(arg='/Users/jacobelias/Library/Jupyter/runtime/kernel-340c986b-9a7c-4f18-bb1b-95c776280545.json', flag=True, ame=None, week=None, verbose=0)


Week:  2
J.G.'s  ZZ's 
Chiraqui Soldiers
Top G
Begelbaum Donuts
Kupp My  Bells Gurley


In [154]:
def biggestBlowOut(league, week):
    outcomes = []
    
    box_scores = league.box_scores(week)
    
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        diff = 0
        if matchup.home_score >= matchup.away_score:
            diff = matchup.home_score - matchup.away_score
            outcomes.append((matchup, diff))
        else:
            diff = matchup.away_score - matchup.home_score
            outcomes.append((matchup, diff))
            
    sorted_scores = sorted(outcomes, key=lambda tup: tup[1], reverse=True)

    return sorted_scores


blowoutMatchup = biggestBlowOut(league, week)[0]

print("======== Biggest Blowout (", blowoutMatchup[1], ") =========")
print(blowoutMatchup[0].home_team.team_name, blowoutMatchup[0].home_score, " vs ", blowoutMatchup[0].away_team.team_name, blowoutMatchup[0].away_score )


======== Biggest Blowout ( 42.7 ) =========
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


In [157]:
def biggestBenchWarmer(league, week, position):
    if position not in POSITIONS:
        return []
    
    
    benchWarmers = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        for player in matchup.home_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
                benchWarmers.append((player.points,player.name,player.position,matchup.home_team.team_name))
                
                
        for player in matchup.away_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.away_team, " ",player.points )
                benchWarmers.append((player.points,player.name,player.position,matchup.away_team.team_name))
    

    return sorted(benchWarmers, key=lambda tup: tup[0], reverse=True)


def prettyPrintBenchWarmers(benchWarmers, position):
    print("=============== ", position, " ===============")
    for index, warmer in enumerate(benchWarmers):
        print(index+1,"", warmer[0], " ", warmer[1], " ", warmer[3])
    print("  ")
        

### QBs
qbWarmers = biggestBenchWarmer(league, week, "QB")
prettyPrintBenchWarmers(qbWarmers[:5], "QB")

### RBs
rbWarmers = biggestBenchWarmer(league, week, "RB")
prettyPrintBenchWarmers(rbWarmers[:5], "RB")

### TEs
teWarmers = biggestBenchWarmer(league, week, "TE")
prettyPrintBenchWarmers(teWarmers[:5], "TE")
        
    

===============  QB  ===============
1  26.7   Carson Wentz   Gisele'd in my  Wentz
2  14.4   Marcus Mariota   Rosen Sandler
3  12.6   Mac Jones   The  Red Army
4  10.3   Mitch Trubisky   J.G.'s  ZZ's 
5  7.6   Jameis Winston   Top G
  
===============  RB  ===============
1  17.8   Tony Pollard   Top G
2  15.7   Damien Harris   Chiraqui Soldiers
3  14.6   David Montgomery   Kupp My  Bells Gurley
4  12.5   Breece Hall   Top G
5  10.7   Darrell Henderson Jr.   El Chapo
  
===============  TE  ===============
1  14.0   Darren Waller   Gisele'd in my  Wentz
2  10.7   Dallas Goedert   J.G.'s  ZZ's 
3  4.1   T.J. Hockenson   Gisele'd in my  Wentz
4  2.8   Dalton Schultz   Begelbaum Donuts
5  0.0   George Kittle   El Chapo
  


In [128]:
# ### Total Points Played / Optimal Lineup
# lineup_format = {
#     "QB": 2
#     "RB": 2
#     ""
# }
# def excessBenchPoints(lineup, position):
#     for player in lineup:
#         if player.position == position and player.slot_position == "BE":
# #              print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
#         benchWarmers.append((player.points,player.name,player.position,matchup.home_team))
        
    



# def managerEffencieny(league, week):
    
#     for box_scores = league.box_scores(week)
#         for matchup in box_scores:
#             getOptimalLineup(matchup.home_lineup)
    

In [162]:
def worstWin(league, week):
    box_scores = league.box_scores(week)
    lowestWinningScore = 99999999999
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            lowestWinningScore = min(matchup.home_score, lowestWinningScore)
        else:
            lowestWinningScore = min(matchup.away_score, lowestWinningScore)
            
        
        if lowestWinningScore == matchup.away_score or lowestWinningScore == matchup.home_score:
            saved = matchup
    print("================== Worst Win ===============")      
    print(saved.home_team.team_name, saved.home_score, " vs ", saved.away_team.team_name, saved.away_score)

    
worstWin(league, week)

================== Worst Win ===============
J.G.'s  ZZ's  89.8  vs  The  Red Army 83.6


In [161]:
def worstLoss(league, week):
    box_scores = league.box_scores(week)
    higestLoss = 0
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            higestLoss = max(matchup.away_score, higestLoss)
        else:
            higestLoss = max(matchup.home_score, higestLoss)
            
        
        if higestLoss == matchup.away_score or higestLoss == matchup.home_score:
            saved = matchup
    print("================== Worst Loss ===============")
    print(saved.home_team.team_name, saved.home_score, " vs ", saved.away_team.team_name, saved.away_score)

    
worstLoss(league, week)

================== Worst Loss ===============
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


In [166]:
def startingLineupAverage(lineup):
    starterOutput = [] 
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            starterOutput.append(player.points)         
    return sum(starterOutput)/len(starterOutput)


def highestScoringStarter(lineup):
    highestScore = 0
    bestPlayer = None
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            if player.points > highestScore:
                highestScore = player.points
                bestPlayer = player
    return bestPlayer
    
    
## Team with greatest differences between max player and team average
def topHeavyTeams(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        maxPlayer = highestScoringStarter(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[3], reverse=True)
    return allTeamSorted
                                         

topHeavyList = topHeavyTeams(league, week)

print("======= Top Heavy Teams (Carried by one player) ========")
for index, team in enumerate(topHeavyList[:3]):
    print(index+1, " ", team[0], team[2].name, team[2].points, "{:.2f}".format(team[1]))
        

======= Top Heavy Teams (Carried by one player) ========
1   Rosen Sandler Tua Tagovailoa 38.7 12.12
2   Top G Stefon Diggs 38.8 13.07
3   El Chapo Lamar Jackson 42.5 20.24


In [169]:
## Team with greatest differences between max player and team average
def highestTeamAverageForStarters(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[1], reverse=True)
    return allTeamSorted


print("======= All around hitters ( Highest team Average points)  ========")
for index, team in enumerate(highestTeamAverageForStarters(league,week)[:3]):
    print(index+1, " ", team[0], team[1])
                                   

======= All around hitters ( Highest team Average points)  ========
1   El Chapo 20.2375
2   Gisele'd in my  Wentz 16.1125
3   Grayson BeatYu 15.375


###########  Week 2 Output  #################
======== Biggest Blowout ( 42.7 ) =========
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


######## Biggest Bench Warmers (Highest Points left on bench ) ###########
===============  QB  ===============
1  26.7   Carson Wentz     Gisele'd in my  Wentz
2  14.4   Marcus Mariota   Rosen Sandler
3  12.6   Mac Jones   The  Red Army
4  10.3   Mitch Trubisky   J.G.'s  ZZ's 
5  7.6    Jameis Winston   Top G
  
===============  RB  ===============
1  17.8   Tony Pollard            Top G
2  15.7   Damien Harris           Chiraqui Soldiers
3  14.6   David Montgomery        Kupp My  Bells Gurley
4  12.5   Breece Hall             Top G
5  10.7   Darrell Henderson Jr.   El Chapo
  
===============  TE  ===============
1  14.0   Darren Waller   Gisele'd in my  Wentz
2  10.7   Dallas Goedert  J.G.'s  ZZ's 
3  4.1   T.J. Hockenson   Gisele'd in my  Wentz
4  2.8   Dalton Schultz   Begelbaum Donuts
5  0.0   George Kittle    El Chapo
  
=================================================

######### Worst Win (Lowest Scoring Winner) ################
J.G.'s  ZZ's  89.8  vs  The  Red Army 83.6

######### Worst Loss (Highest Scoring Loser) ################
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


########## Top Heavy Teams ################
*: biggest difference between top player and team average
     
     Team        |  Player  |   Top score  | Team Avg
1   Rosen Sandler  Tua Tagovailoa  38.7          12.12
2   Top G          Stefon Diggs    38.8          13.07
3   El Chapo Lamar Jackson         42.5          20.24



######## All around hitters ( Highest team Average points)  ################
1   El Chapo               20.2375
2   Gisele'd in my  Wentz  16.1125
3   Grayson BeatYu         15.375

